In [1]:
import requests
import itchat
from itchat.content import *
# Import necessary modules
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config


session = None
username = None
password = None
headers = None
auth_token = None
positions = None

##############################
#Logging in and initializing
##############################

def __init__(self):
    self.session = requests.session()
    try:
        self.session.proxies = urllib.getproxies() #py2
    except:
        self.session.proxies = urllib.request.getproxies() #py3
    self.headers = {
        "Accept": "*/*",
        "Accept-Encoding": "gzip, deflate",
        "Accept-Language": "en;q=1, fr;q=0.9, de;q=0.8, ja;q=0.7, nl;q=0.6, it;q=0.5",
        "Content-Type": "application/x-www-form-urlencoded; charset=utf-8",
        "X-Robinhood-API-Version": "1.0.0",
        "Connection": "keep-alive",
        "User-Agent": "Robinhood/823 (iPhone; iOS 7.1.2; Scale/2.00)"
    }
    self.session.headers = self.headers

def login(self, username, password, mfa_code=None):
    self.username = username
    self.password = password
    self.mfa_code = mfa_code
    #fields = { 'password' : self.password, 'username' : self.username, 'mfa_code': self.mfa_code }
    #fields = { 'password' : self.password, 'username' : self.username }
    
    if mfa_code:
        fields = { 'client_id' : 'c82SH0WZOsabOXGP2sxqcj34FxkvfnWRZBKlBjFS',
                    'expires_in' : 86400,
                    'grant_type': 'password',
                    'password' : self.password,
                    'scope' : 'internal',
                    'username' : self.username,
                    'mfa_code': self.mfa_code }
    else:
        fields = { 'client_id' : 'c82SH0WZOsabOXGP2sxqcj34FxkvfnWRZBKlBjFS',
                    'expires_in' : 86400,
                    'grant_type': 'password',
                    'password' : self.password,
                    'scope': 'internal',
                    'username' : self.username }
    try:
        data = urllib.urlencode(fields) #py2
    except:
        data = urllib.parse.urlencode(fields) #py3
    res = self.session.post(self.endpoints['login'], data=data)
    #res.raise_for_status()
    res = res.json()
    #print(data)
    try:
        #self.auth_token = res['token']
        self.oauth_token = res['access_token']
    except KeyError:
        return res
    #self.headers['Authorization'] = 'Token '+self.auth_token
    self.headers['Authorization'] = 'Bearer ' + self.oauth_token
    return True

# Create a trainer that uses this config
trainer = Trainer(config.load("C:\\Users\\jiawe\\Downloads\\chatbot-master\\chatbot-master\\rasa_nlu_chi\\sample_configs\\config_jieba_mitie_sklearn.yml"))

# Load the training data
training_data = load_data('C:\\Users\\jiawe\\Downloads\\chatbot-master\\chatbot-master\\rasa_nlu_chi\\data\\examples\\rasa\\stock-rasa_ch.json')

# Create an interpreter by training the model
interpreter = trainer.train(training_data)

def en_ch_d(english_name):
    transplate_dict={'volume':'成交量',
                     'market_cap':'市值',
                     'high':'最高股价',
                     'last_trade_price':'当前股价'
                    }
    return transplate_dict[english_name]

def en_ch_c(english_name):
    transplate_dict={'F':'福特',
                     'FB':'脸书',
                     'AAPL':'苹果',
                     'MSFT':'微软'
                    }
    return transplate_dict[english_name]

@itchat.msg_register(TEXT)
def respond(msg):
    global response_dict
    global company_name
    times_comp=0
    times_det=0
    message=msg['Text']
    entities = interpreter.parse(message)["entities"]
    for ents in entities:
        if ents['entity']=='company':
            comp=ents['value']
            url='https://api.robinhood.com/instruments/?symbol={}'.format(comp)
            r=requests.get(url)
            print(url)
            response_dict1=r.json()
            print(response_dict1)
            print('*********')
            url='https://api.robinhood.com/fundamentals/{}/'.format(comp)
            r=requests.get(url)
            print(r)
            response_dict2=r.json()
            print(response_dict2)
            print('*********')            
            response_dict=dict(response_dict1,**response_dict2)
            company_name=en_ch_c(comp)
            times_comp=1
            for ents in entities:
                if ents['entity']=='detail':
                    detail=ents['value']
                    detail_name=en_ch_d(detail)
                    print(detail_name)
                    response="{0}的{1}是:{2}".format(company_name, detail_name, response_dict[detail])
                    times_det=1
                    itchat.send(response, toUserName=msg['FromUserName'])
            if times_det==0:
                response="您想知道关于这个公司的什么信息?"
                itchat.send(response, toUserName=msg['FromUserName'])
    
    if times_comp==0:
        for ents in entities:
            detail=ents['value']
            detail_name=en_ch_d(detail)
            response="{0}的{1}是:{2}".format(company_name, detail_name, response_dict[detail])
            itchat.send(response, toUserName=msg['FromUserName'])

itchat.auto_login()
itchat.run()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\jiawe\AppData\Local\Temp\jieba.cache
Loading model cost 1.492 seconds.
Prefix dict has been built succesfully.


Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.0s finished
Getting uuid of QR code.
Please scan the QR code to log in.
Please press confirm on your phone.
Loading the contact, this may take a little while.
Login successfully as 🌝
Start auto replying.


https://api.robinhood.com/instruments/?symbol=MSFT
{'previous': None, 'results': [{'margin_initial_ratio': '0.5000', 'rhs_tradability': 'tradable', 'id': '50810c35-d215-4866-9758-0ada4ac79ffa', 'market': 'https://api.robinhood.com/markets/XNAS/', 'simple_name': 'Microsoft', 'min_tick_size': None, 'maintenance_ratio': '0.2500', 'tradability': 'tradable', 'state': 'active', 'type': 'stock', 'tradeable': True, 'fundamentals': 'https://api.robinhood.com/fundamentals/MSFT/', 'quote': 'https://api.robinhood.com/quotes/MSFT/', 'symbol': 'MSFT', 'day_trade_ratio': '0.2500', 'name': 'Microsoft Corporation Common Stock', 'tradable_chain_id': '1ac71e01-0677-42c6-a490-1457980954f8', 'splits': 'https://api.robinhood.com/instruments/50810c35-d215-4866-9758-0ada4ac79ffa/splits/', 'url': 'https://api.robinhood.com/instruments/50810c35-d215-4866-9758-0ada4ac79ffa/', 'country': 'US', 'bloomberg_unique': 'EQ0010174300001000', 'list_date': '1987-09-17'}], 'next': None}
*********
<Response [401]>


Traceback (most recent call last):
  File "C:\Users\jiawe\Anaconda3\lib\site-packages\itchat\components\register.py", line 66, in configured_reply
    r = replyFn(msg)
  File "<ipython-input-1-6e87abf63ffe>", line 124, in respond
    response_dict2=r.json()
  File "C:\Users\jiawe\Anaconda3\lib\site-packages\requests\models.py", line 892, in json
    return complexjson.loads(self.text, **kwargs)
  File "C:\Users\jiawe\Anaconda3\lib\site-packages\simplejson\__init__.py", line 518, in loads
    return _default_decoder.decode(s)
  File "C:\Users\jiawe\Anaconda3\lib\site-packages\simplejson\decoder.py", line 370, in decode
    obj, end = self.raw_decode(s)
  File "C:\Users\jiawe\Anaconda3\lib\site-packages\simplejson\decoder.py", line 400, in raw_decode
    return self.scan_once(s, idx=_w(s, idx).end())
simplejson.errors.JSONDecodeError: Expecting value: line 1 column 1 (char 0)

Traceback (most recent call last):
  File "C:\Users\jiawe\Anaconda3\lib\site-packages\urllib3\connectionpool.py"

Traceback (most recent call last):
  File "C:\Users\jiawe\Anaconda3\lib\site-packages\urllib3\connection.py", line 141, in _new_conn
    (self.host, self.port), self.timeout, **extra_kw)
  File "C:\Users\jiawe\Anaconda3\lib\site-packages\urllib3\util\connection.py", line 60, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "C:\Users\jiawe\Anaconda3\lib\socket.py", line 745, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno 11001] getaddrinfo failed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\jiawe\Anaconda3\lib\site-packages\urllib3\connectionpool.py", line 601, in urlopen
    chunked=chunked)
  File "C:\Users\jiawe\Anaconda3\lib\site-packages\urllib3\connectionpool.py", line 346, in _make_request
    self._validate_conn(conn)
  File "C:\Users\jiawe\Anaconda3\lib\site-packages\urllib3\connect

Traceback (most recent call last):
  File "C:\Users\jiawe\Anaconda3\lib\site-packages\urllib3\connection.py", line 141, in _new_conn
    (self.host, self.port), self.timeout, **extra_kw)
  File "C:\Users\jiawe\Anaconda3\lib\site-packages\urllib3\util\connection.py", line 60, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "C:\Users\jiawe\Anaconda3\lib\socket.py", line 745, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno 11001] getaddrinfo failed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\jiawe\Anaconda3\lib\site-packages\urllib3\connectionpool.py", line 601, in urlopen
    chunked=chunked)
  File "C:\Users\jiawe\Anaconda3\lib\site-packages\urllib3\connectionpool.py", line 346, in _make_request
    self._validate_conn(conn)
  File "C:\Users\jiawe\Anaconda3\lib\site-packages\urllib3\connect

LOG OUT!
